### Construindo Arvore

In [53]:
class Stack(list):
    def push(self, __object):
        self.append(__object)

    def isempty(self): return not self

class Tree:

    COUNT = [10]

    def __init__(self, value, left: 'Tree' = None, right: 'Tree' = None):
        self.value = value
        self._left: Tree = self.__totree(left) if left else None 
        self._right: Tree = self.__totree(right) if right else None
        

    @staticmethod
    def __totree(__object: 'Tree'): return __object if isinstance(__object, Tree) else Tree(__object)


    @staticmethod
    def _clone_tree(root: 'Tree'):
        if root is None:
            return None
        copy = Tree(root.value)
        if root.left:
            copy.left = Tree._clone_tree(root.left) 
        if root.right:
            copy.right = Tree._clone_tree(root.right)
        return copy


    def printtree(self, sep='\n'):
        print(self.value)
        if self.left:
            self.left.printtree()
        if self.right:
            self.right.printtree()

    @property
    def left(self) -> 'Tree': return self._left

    @property
    def right(self) -> 'Tree': return self._right

    @left.setter
    def left(self, other: 'Tree'):
        self._left = self.__totree(other) 

    @right.setter
    def right(self, other: 'Tree'):
        self._right = self.__totree(other)


    def bshow(self):
        return self._bshow(self, '')

    
    @staticmethod
    def _bshow(node: 'Tree', heritage: str):
        if node and (node.left or node.right):
            Tree._bshow(node.right, heritage + "r")
        
        i = 0
        while i < (len(heritage) - 1):
            print("│   " if heritage[i] != heritage[i+1] else "    ", end='')
            i += 1
        
        if heritage != '':
            print("┌───" if heritage[-1] == 'r' else "└───", end='')
        
        if not node:
            print()
            return

        print(node.value)

        if node and (node.left or node.right):
            Tree._bshow(node.left, heritage + "l")

In [54]:
root = Tree(2, 3, 5)

In [55]:
root.bshow()

┌───5
2
└───3


In [56]:
another = Tree._clone_tree(root)

In [57]:
another.bshow()

┌───5
2
└───3


### Algoritmo de construção da arvore

In [44]:
#input_ = 'MEM(+(CONST 1,CONST 2))'
#input_ = 'MOVE(MEM(+(MEM(+(FP,CONST a)),*(TEMP i,CONST 4))),MEM(+(FP,CONST x)))'
# EXP FINAL
#------------------------------------------
# MOVE(
#     MEM(
#         +(
#             MEM(
#                 +(
#                     FP,
#                     CONST a
#                 )
#             ),
#             *(
#                 TEMP i,
#                 CONST 4
#             )
#         )
#     ),
#     MEM(
#         +(
#             FP,
#             CONST x
#         )
#     )
# )

In [58]:
def from_expression(input_: str, verbose=False) -> 'Tree':
        main_stack = Stack()
        worker_stack = Stack()
        op = ''

        def unstack():
            while not main_stack.isempty():
                son = main_stack.pop()
                if son == '(':
                    father = main_stack.pop()
                    left = worker_stack.pop() if worker_stack else None
                    right = worker_stack.pop() if worker_stack else None

                    worker_stack.push(Tree(value=father, left=left, right=right))
                    break
                worker_stack.push(son)
            
            exp = worker_stack.pop()
            main_stack.push(exp)
            
            if verbose: exp.bshow()

        for char in input_:    
            if char in ['(', ',', ')']:
                if op: main_stack.push(op)
                if char == '(': main_stack.push(char)
                elif char == ')': unstack()
                op = '' 
            else: op += char  
        return main_stack.pop()

In [59]:
inputs = [
    'MEM(+(CONST 1,CONST 2))',
    'MOVE(MEM(+(MEM(+(FP,CONST a)),*(TEMP i,CONST 4))),MEM(+(FP,CONST x)))'
]

In [60]:
expression_trees = map(from_expression, inputs)

In [61]:
for idx, tree in enumerate(expression_trees):
    print(f'Arvore {idx+1} :')
    tree.bshow()
    print()

Arvore 1 :
┌───
MEM
│   ┌───CONST 2
└───+
    └───CONST 1

Arvore 2 :
    ┌───
┌───MEM
│   │   ┌───CONST x
│   └───+
│       └───FP
MOVE
│   ┌───
└───MEM
    │       ┌───CONST 4
    │   ┌───*
    │   │   └───TEMP i
    └───+
        │   ┌───
        └───MEM
            │   ┌───CONST a
            └───+
                └───FP



### Fase de seleção de instrução